In [1]:
import numpy as np
import pandas as pd
import math
import re

In [2]:
#FILE_NAME = "test.txt"
FILE_NAME = "input.txt"
DIAG=False
DIAG2 = False

In [4]:
def buildRulesAndPartsFromLines(lines):
    
    cur_x = 0
    cur_y = 0
    
    map_df = pd.DataFrame(["#"])
    
    boundary = -1
    for i,line in enumerate(lines):
        if line[-1]=="\n": line = line[:-1]
        line = line.strip()        
        if len(line)==0: 
            boundary = i
    
    lines_rules = lines[:boundary]
    lines_gears = lines[boundary+1:]
    
    rules = buildRulesFromLines(lines_rules)
    gears,gears_outst_rules = buildGearsFromLines(lines_gears)
                    
    return rules,gears,gears_outst_rules
    

In [6]:
def evalRuleOnObject(rule,gear):
    if DIAG: print(rule)
    rule_cond,rule_outcome = rule
    if not rule_cond=="True":
        #do the replacement
        if DIAG: print(rule_cond)
        mmm = re.match("^([a-zA-Z]+)",rule_cond)        
        gear_var = mmm[0]
    else:
        gear_var = None
        
    if not gear_var is None:
        if DIAG:
            print(gear)
            print(gear_var)
        rule_cond = rule_cond.replace(gear_var,gear[gear_var],1)
    
    if eval(rule_cond):
        return (True,rule_outcome)
    else:
        return (False,None)        
        
    return None

evalRuleOnObject(('a>716', 'R'),{'x': '787', 'm': '2655', 'a': '1222', 's': '2876'})


(True, 'R')

In [7]:
def buildRulesFromLines(lines_rules):    
    rules = {}
    for line in lines_rules:
        z=line.find("{")
        name=line[:z]
        line=line[z+1:].strip()[:-1]
        
        
        res = []
        for x in line.split(","):            
            tmp = x.split(":")            
            if len(tmp)>1:
                res.append((tmp[0],tmp[1]))
            else:
                res.append(("True",tmp[0]))
        rules[name]=res
    return rules
               
        
y=["px{a<2006:qkq,m>2090:A,rfg}","pv{a>1716:R,A}","lnx{m>1548:A,A}"]
buildRulesFromLines(y)

{'px': [('a<2006', 'qkq'), ('m>2090', 'A'), ('True', 'rfg')],
 'pv': [('a>1716', 'R'), ('True', 'A')],
 'lnx': [('m>1548', 'A'), ('True', 'A')]}

In [8]:
def buildGearsFromLines(lines_gears):    
    gears = {}
    gears_vs_rules = {}
    for i,line in enumerate(lines_gears):
        line=line.strip()
        gear = {}
        for x in line[1:-1].split(","):
            feat,val = x.split("=")
            gear[feat]=val
            
            gears[i]=gear
            gears_vs_rules[i]=["in"]
    return gears,gears_vs_rules
               
        
y=["{x=787,m=2655,a=1222,s=2876}","{x=1679,m=44,a=2067,s=496}","{x=2036,m=264,a=79,s=2244}","{x=2461,m=1339,a=466,s=291}","{x=2127,m=1623,a=2188,s=1013}"]
buildGearsFromLines(y)

({0: {'x': '787', 'm': '2655', 'a': '1222', 's': '2876'},
  1: {'x': '1679', 'm': '44', 'a': '2067', 's': '496'},
  2: {'x': '2036', 'm': '264', 'a': '79', 's': '2244'},
  3: {'x': '2461', 'm': '1339', 'a': '466', 's': '291'},
  4: {'x': '2127', 'm': '1623', 'a': '2188', 's': '1013'}},
 {0: ['in'], 1: ['in'], 2: ['in'], 3: ['in'], 4: ['in']})

In [9]:
def processPart(part,rule_name,rules):
    """
    part_outst_rules = [(),(),()]
    """
    part_outst_rules = rules[rule_name]
    while len(part_outst_rules) > 0:
        rule = part_outst_rules[0]
        part_outst_rules = part_outst_rules[1:]
        
        if DIAG: print("processPart -- part =",part)
        switch_over,new_rule = evalRuleOnObject(rule,part)
        if new_rule == "R":
            return False
        elif new_rule == "A":
            return True
        elif not switch_over:
            pass
        else:
            # switch over
            return processPart(part,new_rule,rules)
    
    print("Didn't reach conculsion")
    return False     
    
        

In [10]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    rules,parts,parts_outst_rules = buildRulesAndPartsFromLines(lines)
   
    
    
    return rules,parts,parts_outst_rules
    

In [11]:
def score(part):
    res = 0
    for k,v in part.items():
        res+=int(v)
    return res

In [12]:
rules,parts,parts_outst_rules=parseFile(FILE_NAME)
if DIAG:
    print(rules)
    print("--------")
    print(parts)
    print("--------")
    print(parts_outst_rules)
res = 0
for part_num,part in parts.items():
    decision = processPart(part,"in",rules)
    part_score = score(part)
    if decision: res+=part_score
    if DIAG2: print("Part: {}, decision={}, score={}",part,decision,part_score)
print(res)

333263


# Trying things out

In [13]:
evalRuleOnObject(rules["px"][0],parts[0])

(False, None)

In [3]:
eval("12<20000")


True

In [5]:
eval("True")

True

In [14]:
rules["px"][0]

('m<2194', 'A')

In [15]:
parts[0].copy()

{'x': '1691', 'm': '2832', 'a': '1861', 's': '2722'}